#### from functools import cache

## cache / lru_cache:
- 记录以往函数运行的结果，避免参数重复时反复调用，达到提高性能的作用，相同参数会调用缓存

```
maxsize : 最大缓存个数，超过时将最近最少被访问的元素移除，从而腾出空间给新的元素

type :true:不同类型的参数分别缓存，如3.0和3  False:3.0和3都是同一个

```

In [7]:
from functools import lru_cache


@lru_cache(maxsize=None)
def test_(a,b):
    print("已经被调用")

print(test_(1,2))
print(test_(1,2)) # 第二次调用缓存


test_list=[]
@lru_cache(maxsize=None)
def test_value(val):
    test_list.append(val)
    print(f'test_list append :{val}')
    for index,value in enumerate(test_list):
        print(index,value)

test_value(2)
test_value(2) # 第二次直接调用缓冲运行的结果，而不会再次执行append
print(test_list)

已经被调用
None
None
test_list append :2
0 2
[2]
